# Calculate USA zip code level 3G coverage data
* Jie XIA SUSTech
* 2025-02-16

# 1. Check the zip code shapefile
* find reigon name and region id columns

In [ ]:
import geopandas as gpd
import pandas as pd


def check_shp_head(shp_path):

    gdf = gpd.read_file(shp_path)
    print(gdf.head())

    return gdf


zip_shp_path = r"E:\2023 USA zip code shapefile\Usa_zip_code_2023.shp"

gdf_zip = check_shp_head(zip_shp_path)


# 2. generate intermediate file
* generate reproject USA zip code areas shp
* generate masked, reproject, and resample USA zip code areas population tif (from global)

In [ ]:
import os
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling

def reproject_shapefile(input_shapefile, target_crs, save_path):
    """
    Using geopandas to reproject boundary vector file to targer CRS
    """
    print(f"Start to reporject: {os.path.basename(input_shapefile)}")

    gdf = gpd.read_file(input_shapefile)
    original_crs = gdf.crs
    gdf = gdf.to_crs(target_crs)
    reproject_crs = gdf.crs
    gdf.to_file(save_path, driver='ESRI Shapefile')

    print(f"finished reprojecting.\nThe original CRS is: {original_crs}, now the CRS is: {reproject_crs}\n")

def reproject_and_resample_raster(input_raster, output_raster, target_crs, resolution):
    """
    Using rasterio to:
        reproject and resample the raster file to target CRS and resolution
    """
    with rasterio.open(input_raster) as src:
        print(f"Start to process input raster file '{os.path.basename(input_raster)}'")

        # Retrieve the bounds of the input raster
        left, bottom, right, top = src.bounds

        # Calculate target reproject transform matrix and target shape(i.e. height and width)
        transform, width, height = calculate_default_transform(
            src.crs,               # CRS of source raster
            target_crs,           # CRS of target to reproject
            src.width,
            src.height,
            left=left,
            bottom=bottom,
            right=right,
            top=top,
            resolution=(resolution, resolution)
        )

        # Update metadata of input raster
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # Create output raster file and write data
        with rasterio.open(output_raster, 'w', **kwargs) as dst:
            print(f"Start to write output file '{os.path.basename(output_raster)}'")

            for i in range(1, src.count + 1): # iterate all bands
                reproject(
                    source=rasterio.band(src, i),      
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=dst.transform,
                    dst_crs=target_crs,
                    resampling=Resampling.bilinear
                )
            print(f"Finished writing output raster file '{output_raster}' \n")

def mask_raster_with_shapefile(mask_shapefile, input_raster, output_raster):
    """
    Using rasterio and geopandas to:
        mask the raster to the regional level
    """
    with rasterio.open(input_raster) as src:
        print(f"Start to mask raster file '{os.path.basename(input_raster)}'")
        raster_shape = src.shape
        gdf = gpd.read_file(mask_shapefile)
        regional_boundary = [feature["geometry"] for feature in gdf.iterfeatures()]

        out_image, out_transform = mask(src, 
                                        regional_boundary, 
                                        crop=True, # mask to the minimum range of raster
                                        nodata=0)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "nodata": 0
        })
    with rasterio.open(output_raster, "w", **out_meta) as dest:
        dest.write(out_image)
    print(f"Save masked raster at '{output_raster}'")
    print(f"The original raster shape is: {raster_shape}, After masking the shape is: {out_image.shape} \n")
    return output_raster

if __name__ == "__main__":

    ### Setting saving folder
    generate_data_folder = r"E:\3G coverage population rate\2008-2016 USA zip code 3G coverage\generate data"

    ### Reproject boundary shp
    zip_area_shp = r"E:\2023 USA zip code shapefile\Usa_zip_code_2023.shp"
    zip_save_path = os.path.join(generate_data_folder, "reproject_us_zip_code.shp")
    target_crs = "EPSG:8857"   

    reproject_shapefile(zip_area_shp, target_crs, zip_save_path)

    ### Reproject and resample global population tif
    global_pop_raster = r"E:\2000-2020 NASA population density raster\gpw-v4-population-density-rev11_2000_30_sec_tif\gpw_v4_population_density_rev11_2000_30_sec.tif"
    rep_and_res_pop_raster = os.path.join(generate_data_folder, "rep_res_global_pop_2000.tif")
    target_crs = "EPSG:8857"
    resolution = 1000
    reproject_and_resample_raster(global_pop_raster, rep_and_res_pop_raster, target_crs, resolution)

    ### Mask pop raster to CBSA
    mask_shapefile = zip_save_path
    input_raster = rep_and_res_pop_raster
    zip_code_pop_raster = os.path.join(generate_data_folder, "us_zip_code_pop_2000.tif")
    mask_raster_with_shapefile(mask_shapefile, input_raster, zip_code_pop_raster)

# 3. Generate zip code areas population weight tif
* then align 3G coverage data to zip code areas weight tif (use multi-thread to accelerate)

In [ ]:
import os
import numpy as np
import rasterio
import geopandas as gpd
from rasterio.features import rasterize
from rasterio.warp import reproject, Resampling
from concurrent.futures import ThreadPoolExecutor, as_completed

def run_multithreaded_tasks(task_function, task_args_list, max_workers=8):

    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(task_function, *args): args for args in task_args_list}
        for future in as_completed(futures):
            args = futures[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"Task {args} failed: {e}")

        executor.shutdown(wait=True)

    return results

def calculate_weight_raster_parallel(
    reprojected_regional_boundary,
    masked_population_raster,
    output_weight_raster,
    region_column_name="NAME"
):
    """
    Compute the 'multi-region weight raster' using rasterization + zonal statistics.
    """
    # Read population raster metadata
    with rasterio.open(masked_population_raster) as pop_src:
        pop_data = pop_src.read(1)
        pop_meta = pop_src.meta.copy()
        # Convert the output to float32 and set nodata to 0
        pop_meta.update({"dtype": "float32", "nodata": 0})
        height, width = pop_data.shape
        pop_transform = pop_src.transform

    # Read the Shapefile
    gdf = gpd.read_file(reprojected_regional_boundary)
    print("First five polygons read:")
    print(gdf.head(5))

    # Check if the column exists
    if region_column_name not in gdf.columns:
        raise ValueError(f"Column '{region_column_name}' not found in shapefile.")

    # Generate region_id based on region_column_name
    unique_regions = gdf[region_column_name].unique()
    region_name_to_id = {name: idx+1 for idx, name in enumerate(unique_regions)}
    gdf["region_id"] = gdf[region_column_name].map(region_name_to_id)

    # Rasterize region_id
    shapes = ((geom, rid) for geom, rid in zip(gdf.geometry, gdf["region_id"]))
    id_raster = rasterize(
        shapes=shapes,
        out_shape=(height, width),
        transform=pop_transform,
        fill=0,
        dtype='int32'
    )

    # Use np.bincount to calculate total population per region_id
    pop_data_flat = pop_data.ravel()
    id_raster_flat = id_raster.ravel()
    max_id = id_raster_flat.max()
    population_sums = np.bincount(
        id_raster_flat,
        weights=pop_data_flat,
        minlength=max_id + 1
    )

    # Construct the weight raster
    weight_raster = np.zeros_like(pop_data, dtype=np.float32)
    non_zero_mask = (id_raster != 0)
    valid_ids = (population_sums > 0)
    valid_mask = non_zero_mask & valid_ids[id_raster]
    weight_raster[valid_mask] = (
        pop_data[valid_mask] / population_sums[id_raster[valid_mask]]
    )

    # Save as float32
    with rasterio.open(output_weight_raster, "w", **pop_meta) as dst:
        dst.write(weight_raster.astype(np.float32), 1)

    print(f"Weight raster saved at: {output_weight_raster}")


def align_raster_A_to_raster_B(raster_A, raster_B, aligned_raster_save_path, year):
    """
    Align 3G raster to population raster only to avoid redundant alignment calculations.
    """
    if os.path.exists(aligned_raster_save_path):
        print(f"Aligned 3G raster already exists for {year}: {aligned_raster_save_path}")
        return

    with rasterio.open(raster_B) as B_src:
        B_transform = B_src.transform
        B_crs = B_src.crs
        B_shape = (B_src.height, B_src.width)

    with rasterio.open(raster_A) as A_src:
        print(f"Aligning 3G raster for year {year}...")
        meta = A_src.meta.copy()
        meta.update({
            "transform": B_transform,
            "height": B_shape[0],
            "width": B_shape[1],
            "crs": B_crs
        })

        with rasterio.open(aligned_raster_save_path, "w", **meta) as dst:
            for i in range(1, A_src.count + 1):
                reproject(
                    source=rasterio.band(A_src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=A_src.transform,
                    src_crs=A_src.crs,
                    dst_transform=B_transform,
                    dst_crs=B_crs,
                    resampling=Resampling.bilinear
                )

        print(f"Finished alignment for year {year}, saved to {aligned_raster_save_path}")

if __name__ == "__main__":
    generate_data_folder = r"E:\3G coverage population rate\2008-2016 USA zip code 3G coverage\generate data"
    the_3G_coverage_folder = r"E:\3G coverage population rate\2006-2020 Global 3G covered population rate\3G_coverage_data"
    input_shapefile = os.path.join(generate_data_folder, "reproject_us_zip_code.shp")
    input_pop_raster = os.path.join(generate_data_folder, "us_zip_code_pop_2000.tif")
    zip_code_area_weight_raster = os.path.join(generate_data_folder, 'zip_code_pop_weight_raster.tif')

    # Specify the column name to be used
    calculate_weight_raster_parallel(
        reprojected_regional_boundary=input_shapefile,
        masked_population_raster=input_pop_raster,
        output_weight_raster=zip_code_area_weight_raster,
        region_column_name="ZIP_CODE"  # Field name in the shapefile
    )

    ### Align each year’s 3G coverage to the weight raster
    # Use multithreading to align different years' rasters in parallel
    def align_raster_task(the_3G_raster, weight_raster, aligned_3G_raster, year):
        """Thread task: align 3G raster data."""
        align_raster_A_to_raster_B(the_3G_raster, weight_raster, aligned_3G_raster, year)

    # Generate parameter list for all tasks
    task_args_list = [
        (
            os.path.join(the_3G_coverage_folder, str(year), f"rep_and_res_global_3G_cover_{year}.tif"),
            zip_code_area_weight_raster,
            os.path.join(generate_data_folder, f"aligned_3G_coverage_{year}.tif"),
            year
        )
        for year in range(2007, 2018)
    ]

    # Execute alignment in parallel using multithreading
    run_multithreaded_tasks(align_raster_task, task_args_list, max_workers=2)
    print("All 3G coverage rasters aligned successfully.")


# 4. Calculating USA zip code areas 3G covered population rate

Notes:
* This code cannot run in jupyter notebook, since notebook cannot run multiprocessing
* Therefore you should copy it into a .py file to run

In [ ]:
import os
import numpy as np
import geopandas as gpd
import rasterio
import pandas as pd
from rasterstats import zonal_stats
from multiprocessing import Pool

def calculate_regional_3G_covered_population_proportion(args):
    """
    Compute the proportion of the population covered by 3G (batch processing).
    """
    (reprojected_regional_boundary, population_weight, align_3G_raster_path, output_csv, year,
     region_column_name, region_id_column_name) = args

    print(f"Processing year {year}...")

    # **Load population weight data**
    with rasterio.open(population_weight) as popu_src:
        weight_data = popu_src.read(1)
        popu_transform = popu_src.transform

    # **Load 3G coverage data**
    with rasterio.open(align_3G_raster_path) as dst:
        dst_data = dst.read(1)
        dst_data = np.where(dst_data == 2, 1, dst_data)  # Convert weak signal to 1
        dst_data = np.where(dst_data != 1, 0, dst_data)  # Convert all other values to 0

    # **Compute weighted 3G coverage (batch calculation)**
    total_covered_population = weight_data * dst_data

    # **Load regional boundary data**
    gdf = gpd.read_file(reprojected_regional_boundary)

    # **Batch compute zonal statistics**
    stats = zonal_stats(
        gdf,
        total_covered_population,
        affine=popu_transform,
        stats=["sum"],  # Directly compute the weighted covered population
        nodata=np.nan
    )

    # **Organize results**
    results = []
    for i, row in gdf.iterrows():
        region_name = row.get(region_column_name, "Unknown")
        region_id = row.get(region_id_column_name, "Unknown")

        # **Handle None values**
        covered_population_rate = stats[i]["sum"] if stats[i]["sum"] is not None else 0.0

        print(f"Processed {region_name} (ID: {region_id}), {year} Covered rate: {covered_population_rate:.5f}")

        results.append({"Year": year, "Region": region_name, "Region_ID": region_id, "Covered_Population_Rate": covered_population_rate})

    # **Save CSV file**
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"3G coverage results saved to {output_csv}")

    return year, len(gdf)  # Return year and the number of processed regions


if __name__ == "__main__":
    # Set file paths
    result_folder = r"E:\3G coverage population rate\2008-2016 USA zip code 3G coverage\result data"
    generate_data_folder = r"E:\3G coverage population rate\2008-2016 USA zip code 3G coverage\generate data"
    population_weight_tif = os.path.join(generate_data_folder, 'zip_code_pop_weight_raster.tif')
    zip_code_boundary_shp = os.path.join(generate_data_folder, "reproject_us_zip_code.shp")

    # Generate task parameter list
    task_args_list = [
        (
            zip_code_boundary_shp,
            population_weight_tif,
            os.path.join(generate_data_folder, f"aligned_3G_coverage_{year}.tif"),
            os.path.join(result_folder, f"us_zip_code_area_3G_covered_rate_{year}.csv"),
            year,
            "PO_NAME",  # Field name for the region
            "ZIP_CODE",  # Field name for the region ID
        )
        for year in range(2017, 2018)
    ]

    # **Use `multiprocessing` for parallel computation**
    with Pool(processes=6) as pool:  # Use 6 CPU cores in parallel
        results = pool.map(calculate_regional_3G_covered_population_proportion, task_args_list)

    print("All yearly calculations completed.")


# 5. Merge the year-by-year results into panel

In [ ]:
import os
import pandas as pd

# **Set the folder path**
data_folder = r"E:\3G coverage population rate\2008-2016 USA zip code 3G coverage\result data"
output_csv = os.path.join(data_folder, "us_zip_code_3G_coverage_panel.csv")

# **Get all CSV files**
csv_files = [f for f in os.listdir(data_folder) if f.endswith(".csv")]

# **Initialize an empty list to store DataFrames**
df_list = []

# **Iterate and read all CSV files**
for year in range(2007, 2018):
    file_path = os.path.join(data_folder, f"us_zip_code_area_3G_covered_rate_{year}.csv")
    df = pd.read_csv(file_path)

    # Ensure column names are consistent to avoid errors during concatenation
    df.columns = ["Year", "Region", "ZIP_CODE", "Covered_Population_Rate"]

    df_list.append(df)

# **Concatenate all DataFrames**
panel_data = pd.concat(df_list, ignore_index=True)

# **Sort the data**
panel_data.sort_values(by=["ZIP_CODE", "Year"], inplace=True)

# Check for duplicate values
duplicate_count = panel_data.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# **Reset index**
panel_data.reset_index(drop=True, inplace=True)

# **Save as a new CSV file**
panel_data.to_csv(output_csv, index=False, encoding="utf-8-sig")

print(f"Merging completed, data saved to {output_csv}")


In [ ]:
from pandasgui import show

show(output_csv)